In [11]:
import pandas as pd

In [12]:
df=pd.read_csv('Games.csv')

In [13]:
df['release'] = pd.to_datetime(df['release'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [14]:
print(df.head())

   Unnamed: 0                              game           link     release  \
0           0                       Pizza Tower  /app/2231450/  2023-01-26   
1           1                   Resident Evil 4  /app/2050650/  2023-03-24   
2           2  The Murder of Sonic the Hedgehog  /app/2324650/  2023-03-31   
3           3                Pineapple on pizza  /app/2263010/  2023-03-28   
4           4                        Hi-Fi RUSH  /app/1817230/  2023-01-25   

   rating primary_genre                                       store_genres  \
0   96.39    Action (1)                             Action (1), Indie (23)   
1   95.75    Action (1)                         Action (1), Adventure (25)   
2   95.54    Casual (4)  Casual (4), Free to Play (37), Indie (23), Sim...   
3   95.39    Indie (23)          Casual (4), Free to Play (37), Indie (23)   
4   95.09    Action (1)                                         Action (1)   

            publisher        developer  \
0       Tour De Pizz

In [21]:
df.rename(columns={'release': 'release_date'}, inplace=True)
df.rename(columns={'game': 'title'}, inplace=True)
df.rename(columns={'store_genres': 'genres'}, inplace=True)
df.rename(columns={df.columns[0]: 'id'}, inplace=True)
df.rename(columns={'link': 'url'}, inplace=True)
df.rename(columns={'rating' : 'steam_rating'}, inplace=True)


In [22]:
print(df.head())

   id                             title            url release_date  \
0   0                       Pizza Tower  /app/2231450/   2023-01-26   
1   1                   Resident Evil 4  /app/2050650/   2023-03-24   
2   2  The Murder of Sonic the Hedgehog  /app/2324650/   2023-03-31   
3   3                Pineapple on pizza  /app/2263010/   2023-03-28   
4   4                        Hi-Fi RUSH  /app/1817230/   2023-01-25   

   steam_rating primary_genre  \
0         96.39    Action (1)   
1         95.75    Action (1)   
2         95.54    Casual (4)   
3         95.39    Indie (23)   
4         95.09    Action (1)   

                                              genres           publisher  \
0                             Action (1), Indie (23)       Tour De Pizza   
1                         Action (1), Adventure (25)     CAPCOM Co. Ltd.   
2  Casual (4), Free to Play (37), Indie (23), Sim...                SEGA   
3          Casual (4), Free to Play (37), Indie (23)        Majorariat

In [28]:
df['platform_rating'] = 0

In [41]:
import re

In [46]:
def limpiar_cadena(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena
        return re.sub(r'\s*\([^)]*\)', '', texto)
    else:
        return texto  # O podrías retornar una cadena vacía si eso tiene más sentido


In [47]:
df['primary_genre'] = df['primary_genre'].apply(limpiar_cadena)
df['genres'] = df['genres'].apply(limpiar_cadena)


In [48]:
print(df.head())

   id                             title            url release_date  \
0   0                       Pizza Tower  /app/2231450/   2023-01-26   
1   1                   Resident Evil 4  /app/2050650/   2023-03-24   
2   2  The Murder of Sonic the Hedgehog  /app/2324650/   2023-03-31   
3   3                Pineapple on pizza  /app/2263010/   2023-03-28   
4   4                        Hi-Fi RUSH  /app/1817230/   2023-01-25   

   steam_rating primary_genre                                   genres  \
0         96.39        Action                            Action, Indie   
1         95.75        Action                        Action, Adventure   
2         95.54        Casual  Casual, Free to Play, Indie, Simulation   
3         95.39         Indie              Casual, Free to Play, Indie   
4         95.09        Action                                   Action   

            publisher        developer  \
0       Tour De Pizza    Tour De Pizza   
1     CAPCOM Co. Ltd.  CAPCOM Co. Ltd.   
2 

In [49]:
import psycopg2

In [30]:
conn = psycopg2.connect("host=localhost port=5433 dbname=VideoGames user=postgres password=DRESdres951!")

cursor= conn.cursor()

In [50]:
for index, row in df.iterrows():
    try:
        cursor.execute(
            "INSERT INTO games (id, title, url, release_date, primary_genre, genres, steam_rating, platform_rating, publisher, detected_technologies, developer) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            (row['id'], row['title'], row['url'], row['release_date'], row['primary_genre'], row['genres'], row['steam_rating'], row['platform_rating'], row['publisher'], row['detected_technologies'], row['developer'])
        )
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
        continue  # O puedes elegir abortar completamente
    conn.commit()

